In [1]:
# All generated usig ChatGPT and few changes from me
# just an interactive plots for all the plots above with few problems
# to run you need to do the below commands run each one alone in the cell
# pip install --upgrade jupyterlab
# pip install --upgrade ipywidgets
# jupyter labextension update --all
# pip install ipywidgets dash

In [2]:
import dash
from dash import dcc, html
import dash.dependencies
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import io
import base64
from scipy import stats
from math import pi

import seaborn as sns
import pandas as pd

from sklearn.preprocessing import LabelEncoder

In [3]:
df = sns.load_dataset('car_crashes')

,total,speeding,alcohol,not_distracted,no_previous,ins_premium,ins_losses,abbrev
0,18.8,7.332,5.640,18.048,15.040,784.55,145.08,AL
1,18.1,7.421,4.525,16.290,17.014,1053.48,133.93,AK
2,18.6,6.510,5.208,15.624,17.856,899.47,110.35,AZ
3,22.4,4.032,5.824,21.056,21.280,827.34,142.39,AR
4,12.0,4.200,3.360,10.920,10.680,878.41,165.63,CA


In [4]:
le = LabelEncoder()
for i in df.select_dtypes(include='object').columns:
    df[i] = le.fit_transform(df[i])

In [5]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import numpy as np
import base64
import io
import seaborn as sns
import matplotlib.pyplot as plt
from math import pi
from scipy import stats

# Define color palettes for plotting
color_palettes = {
    'Default': px.colors.qualitative.Plotly,
    'Viridis': px.colors.sequential.Viridis,
    'Cividis': px.colors.sequential.Cividis,
    'Inferno': px.colors.sequential.Inferno,
    'Plasma': px.colors.sequential.Plasma,
    'Magma': px.colors.sequential.Magma,
    'Turbo': px.colors.sequential.Turbo,
    'Jet': px.colors.sequential.Jet,
    'Rainbow': px.colors.sequential.Rainbow
}

# Function to create a dropdown component
def create_dropdown(id, options, value, placeholder, multi=False, style=None):
    return dcc.Dropdown(
        id=id,
        options=[{'label': col, 'value': col} for col in options],
        value=value,
        placeholder=placeholder,
        multi=multi,
        style=style
    )

# Function to create an input component
def create_input(id, input_type, value, style=None, **kwargs):
    return dcc.Input(id=id, type=input_type, value=value, style=style, **kwargs)

# Function to create a radio items component
def create_radioitems(id, options, value, style=None):
    return dcc.RadioItems(id=id, options=options, value=value, style=style)

# Function to create a checklist component
def create_checklist(id, options, value, style=None):
    return dcc.Checklist(id=id, options=options, value=value, style=style)

# Function to create an error message component
def create_error_message(message):
    return html.Div([
        html.H4('Error', style={'color': 'red'}),
        html.P(message)
    ], style={'padding': '10px', 'border': '2px solid red', 'border-radius': '5px', 'background-color': '#ffe6e6'})

# Class for creating an interactive plot application
class InteractivePlot:
    def __init__(self, df):
        self.df = df  # DataFrame for plotting
        self.app = dash.Dash(__name__)  # Initialize Dash app
        self.app.layout = self.create_layout()  # Set the layout of the app
        self.register_callbacks()  # Register callbacks for interactivity

    # Function to create the layout of the app
    def create_layout(self):
        columns = [col for col in self.df.columns]  # Get columns from the DataFrame
        return html.Div([
            html.Div([
                html.Div([
                    html.Label("Plot Type"),
                    create_dropdown('plot-type', [
                        'bar', 'box', 'heatmap', 'hexbin', 'histogram', 'pair', 'radar', 'scatter', 'violin'
                    ], 'scatter', "Select a plot type")
                ], style={'padding': '10px'}),
                html.Div([
                    create_dropdown('x-column', columns, None, 'Select X-axis column'),
                    create_dropdown('y-column', columns, None, 'Select Y-axis column', style={'display': 'none'}),
                    create_dropdown('color-column', columns, None, 'Select Color column', style={'display': 'none'}),
                ], style={'padding': '10px'}),
                html.Div([
                    create_input('nbins', 'number', 30, style={'display': 'none'}),
                    create_checklist('kde', [{'label': 'Kernel Density Estimate', 'value': 'kde'}], [], style={'display': 'none'}),
                    create_radioitems('violin-option', [
                        {'label': 'One Column', 'value': 'one'},
                        {'label': 'Two Columns', 'value': 'two'}
                    ], 'one', style={'display': 'none'}),
                    create_radioitems('bar-option', [
                        {'label': 'One Column', 'value': 'one'},
                        {'label': 'Two Columns', 'value': 'two'}
                    ], 'one', style={'display': 'none'}),
                    create_radioitems('box-option', [
                        {'label': 'One Column', 'value': 'one'},
                        {'label': 'Two Columns', 'value': 'two'}
                    ], 'one', style={'display': 'none'}),
                    create_input('num-categories', 'number', 3, style={'display': 'none'}),
                    create_dropdown('category-selector', columns, 
                                    columns[:3], 'Select categories for radar chart', multi=True, style={'display': 'none'}),
                    create_dropdown('color-palette', list(color_palettes.keys()), 'Default', 'Select Color Palette')
                ], style={'padding': '10px'})
            ], style={'width': '20%', 'padding': '20px', 'background-color': '#f0f0f0', 'border-radius': '10px', 'box-shadow': '0 4px 8px rgba(0, 0, 0, 0.1)', 'flex-shrink': '0'}),
            html.Div([
                dcc.Graph(id='plot', style={'width': '100%', 'max-width': '800px'})
            ], style={'flex-grow': '1', 'display': 'flex', 'justify-content': 'center', 'align-items': 'center'}),
            html.Div(id='error-message', style={'width': '100%', 'text-align': 'center', 'margin-top': '20px'})
        ], style={'display': 'flex', 'justify-content': 'center', 'align-items': 'center', 'width': '100%', 'height': '100vh', 'background-color': '#e0f7fa', 'padding': '40px', 'font-family': 'Arial, sans-serif'})

    # Function to register callbacks for interactivity
    def register_callbacks(self):
        @self.app.callback(
            [dash.dependencies.Output('plot', 'figure'),
             dash.dependencies.Output('y-column', 'style'),
             dash.dependencies.Output('color-column', 'style'),
             dash.dependencies.Output('nbins', 'style'),
             dash.dependencies.Output('kde', 'style'),
             dash.dependencies.Output('violin-option', 'style'),
             dash.dependencies.Output('bar-option', 'style'),
             dash.dependencies.Output('box-option', 'style'),
             dash.dependencies.Output('num-categories', 'style'),
             dash.dependencies.Output('category-selector', 'style'),
             dash.dependencies.Output('error-message', 'children')],
            [dash.dependencies.Input('plot-type', 'value'),
             dash.dependencies.Input('x-column', 'value'),
             dash.dependencies.Input('y-column', 'value'),
             dash.dependencies.Input('color-column', 'value'),
             dash.dependencies.Input('nbins', 'value'),
             dash.dependencies.Input('kde', 'value'),
             dash.dependencies.Input('violin-option', 'value'),
             dash.dependencies.Input('bar-option', 'value'),
             dash.dependencies.Input('box-option', 'value'),
             dash.dependencies.Input('num-categories', 'value'),
             dash.dependencies.Input('category-selector', 'value'),
             dash.dependencies.Input('color-palette', 'value')]
        )
        def update_plot(plot_type, x_column, y_column, color_column, nbins, kde, violin_option, bar_option, box_option, num_categories, selected_categories, color_palette):
            try:
                palette = color_palettes[color_palette]
                y_column_style = {'display': 'none'}
                color_column_style = {'display': 'none'}
                nbins_style = {'display': 'none'}
                kde_style = {'display': 'none'}
                violin_option_style = {'display': 'none'}
                bar_option_style = {'display': 'none'}
                box_option_style = {'display': 'none'}
                num_categories_style = {'display': 'none'}
                category_selector_style = {'display': 'none'}
                error_message = None

                # Scatter plot
                if plot_type == 'scatter':
                    fig = px.scatter(self.df, x=x_column, y=y_column, color=color_column, color_discrete_sequence=palette)
                    y_column_style = {'display': 'block'}
                    color_column_style = {'display': 'block'}
                # Bar plot
                elif plot_type == 'bar':
                    if bar_option == 'one':
                        fig = px.bar(self.df, x=x_column, color=x_column, color_discrete_sequence=palette)
                    else:
                        fig = px.bar(self.df, x=x_column, y=y_column, color=y_column, color_discrete_sequence=palette)
                    y_column_style = {'display': 'block'} if bar_option == 'two' else {'display': 'none'}
                    bar_option_style = {'display': 'block'}
                # Violin plot
                elif plot_type == 'violin':
                    if violin_option == 'one':
                        fig = px.violin(self.df, x=x_column, box=True, points='all', color_discrete_sequence=palette)
                    else:
                        fig = px.violin(self.df, x=x_column, y=y_column, box=True, points='all', color_discrete_sequence=palette)
                    y_column_style = {'display': 'block'} if violin_option == 'two' else {'display': 'none'}
                    violin_option_style = {'display': 'block'}
                # Box plot
                elif plot_type == 'box':
                    if box_option == 'one':
                        fig = px.box(self.df, x=x_column, y=y_column, color=color_column, color_discrete_sequence=palette)
                    else:
                        fig = px.box(self.df, x=x_column, y=y_column, color=color_column, color_discrete_sequence=palette)
                    y_column_style = {'display': 'block'}
                    color_column_style = {'display': 'block'}
                    box_option_style = {'display': 'block'}
                # Histogram
                elif plot_type == 'histogram':
                    fig = px.histogram(self.df, x=x_column, nbins=nbins, opacity=0.7, title=f'Histogram of {x_column}', color_discrete_sequence=palette)
                    nbins_style = {'display': 'block'}
                    kde_style = {'display': 'block'}
                    if 'kde' in kde:
                        data = self.df[x_column].dropna()
                        kde_estimator = stats.gaussian_kde(data, bw_method='scott')
                        x = np.linspace(data.min(), data.max(), 1000)
                        kde_y = kde_estimator(x)
                        fig.add_trace(go.Scatter(
                            x=x,
                            y=kde_y * len(data) * (data.max() - data.min()) / nbins,
                            mode='lines',
                            name='KDE',
                            line=dict(color='red', width=2)
                        ))
                # Hexbin plot
                elif plot_type == 'hexbin':
                    fig = self.create_matplotlib_hexbin(x_column, y_column, nbins)
                    y_column_style = {'display': 'block'}
                    nbins_style = {'display': 'block'}
                # Heatmap
                elif plot_type == 'heatmap':
                    fig = self.create_matplotlib_heatmap()
                # Pair plot
                elif plot_type == 'pair':
                    fig = self.create_matplotlib_pairplot()
                # Radar chart
                elif plot_type == 'radar':
                    fig = self.create_matplotlib_radar(selected_categories, num_categories)
                    num_categories_style = {'display': 'block'}
                    category_selector_style = {'display': 'block'}
                # Default to scatter plot if plot type is not recognized
                else:
                    fig = px.scatter(self.df, x=x_column, y=y_column, color_discrete_sequence=palette)
                    y_column_style = {'display': 'block'}

                # Update layout of the figure
                fig.update_layout(
                    title=f'{plot_type.capitalize()} of {x_column} {"and " + y_column if y_column else ""}',
                    xaxis_title=x_column,
                    yaxis_title=y_column if y_column else 'Count'
                )
                return fig, y_column_style, color_column_style, nbins_style, kde_style, violin_option_style, bar_option_style, box_option_style, num_categories_style, category_selector_style, None
            except Exception as e:
                error_message = create_error_message(f"An error occurred: {str(e)}")
                return {}, {}, {}, {}, {}, {}, {}, {}, {}, error_message

    # Function to create a hexbin plot using Matplotlib
    def create_matplotlib_hexbin(self, x_column, y_column, nbins):
        fig, ax = plt.subplots(figsize=(10, 8))
        self.df.plot.hexbin(x=x_column, y=y_column, gridsize=nbins, ax=ax, cmap='viridis')
        buf = io.BytesIO()
        plt.savefig(buf, format='png', bbox_inches='tight')
        buf.seek(0)
        plt.close()
        encoded_image = base64.b64encode(buf.read()).decode('utf-8')
        fig = go.Figure()
        fig.add_layout_image(
            dict(
                source=f'data:image/png;base64,{encoded_image}',
                xref="paper",
                yref="paper",
                x=0,
                y=1,
                sizex=1,
                sizey=1,
                xanchor="left",
                yanchor="top"
            )
        )
        fig.update_layout(width=800, height=600, margin=dict(l=20, r=20, t=20, b=20))
        fig.update_xaxes(visible=False, showticklabels=False)
        fig.update_yaxes(visible=False, showticklabels=False)
        return fig

    # Function to create a heatmap using Matplotlib
    def create_matplotlib_heatmap(self):
        corr = self.df.corr()
        fig, ax = plt.subplots(figsize=(10, 8))
        sns.heatmap(corr, annot=True, ax=ax, cmap='viridis')
        buf = io.BytesIO()
        plt.savefig(buf, format='png', bbox_inches='tight')
        buf.seek(0)
        plt.close()
        encoded_image = base64.b64encode(buf.read()).decode('utf-8')
        fig = go.Figure()
        fig.add_layout_image(
            dict(
                source=f'data:image/png;base64,{encoded_image}',
                xref="paper",
                yref="paper",
                x=0,
                y=1,
                sizex=1,
                sizey=1,
                xanchor="left",
                yanchor="top"
            )
        )
        fig.update_layout(width=800, height=600, margin=dict(l=20, r=20, t=20, b=20))
        return fig

    # Function to create a pair plot using Matplotlib
    def create_matplotlib_pairplot(self):
        sns.pairplot(self.df)
        buf = io.BytesIO()
        plt.savefig(buf, format='png', bbox_inches='tight')
        buf.seek(0)
        plt.close()
        encoded_image = base64.b64encode(buf.read()).decode('utf-8')
        fig = go.Figure()
        fig.add_layout_image(
            dict(
                source=f'data:image/png;base64,{encoded_image}',
                xref="paper",
                yref="paper",
                x=0,
                y=1,
                sizex=1,
                sizey=1,
                xanchor="left",
                yanchor="top"
            )
        )
        fig.update_layout(width=800, height=600, margin=dict(l=20, r=20, t=20, b=20))
        return fig

    # Function to create a radar chart using Matplotlib
    def create_matplotlib_radar(self, selected_categories, num_categories):
        categories = selected_categories[:num_categories] if selected_categories else self.df.columns[:num_categories]
        N = len(categories)
        values = self.df.iloc[0][categories].values.flatten().tolist()
        values += values[:1]
        angles = [n / float(N) * 2 * pi for n in range(N)]
        angles += angles[:1]
        fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))
        ax.fill(angles, values, color='red', alpha=0.25)
        ax.plot(angles, values, color='red', linewidth=2)
        ax.set_yticklabels([])
        ax.set_xticks(angles[:-1])
        ax.set_xticklabels(categories)
        buf = io.BytesIO()
        plt.savefig(buf, format='png', bbox_inches='tight')
        buf.seek(0)
        plt.close()
        encoded_image = base64.b64encode(buf.read()).decode('utf-8')
        fig = go.Figure()
        fig.add_layout_image(
            dict(
                source=f'data:image/png;base64,{encoded_image}',
                xref="paper",
                yref="paper",
                x=0,
                y=1,
                sizex=1,
                sizey=1,
                xanchor="left",
                yanchor="top"
            )
        )
        fig.update_layout(width=800, height=600, margin=dict(l=20, r=20, t=20, b=20))
        return fig

    # Function to run the Dash app
    def run(self):
        self.app.run_server(debug=True)

if __name__ == '__main__':
    plot_app = InteractivePlot(df)
    plot_app.run()


/var/folders/c1/9m9gfrj553l8956nyk7q8gw40000gn/T/ipykernel_75937/888520386.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/var/folders/c1/9m9gfrj553l8956nyk7q8gw40000gn/T/ipykernel_75937/888520386.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [6]:
# http://127.0.0.1:8050/